# **Preprocessing on time series dataset**

In this notebook, I'll perform all the necessary preprocessing for the long term fog-index dataset and also indicate the hidden information about the availabel dataset. **This preprocessed dataset will be saved in the Datasets folder for further prediction purpose.**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


First let's load in our required resources for data loading and model creation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

## Load train data

Here, we'll load the train dataset.

In [ ]:
# for train dataset preprocessing

train = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Original /train_set.csv')

In [ ]:
train.head(5)

,date_time_IST,tmpc,dwpc,relh,drct,wspeedkm,pressure,vsbykm,feelc,cos_min_time,...,3hr_fog,6hr_fog,30min_fog_type,1hr_fog_type,1.5hr_fog_type,2hr_fog_type,2.5hr_fog_type,3hr_fog_type,6hr_fog_type,3hr_onset_cond
0,01-01-2000 00:00,10.0,10.0,100.0,0.0,0.0,101693.2917,0.193121,10.0,1.000000,...,1,1,1,1,1,1,1,1,1,3
1,01-01-2000 00:30,10.0,10.0,100.0,0.0,0.0,101693.2917,0.193121,10.0,0.991077,...,1,1,1,1,1,1,1,1,1,3
2,01-01-2000 01:00,10.0,10.0,100.0,0.0,0.0,101693.2917,0.193121,10.0,0.964469,...,1,1,1,1,1,1,1,1,1,3
3,01-01-2000 01:30,10.0,10.0,100.0,0.0,0.0,101667.8938,0.193121,10.0,0.920650,...,1,1,1,1,1,1,1,1,1,3
4,01-01-2000 02:30,10.0,10.0,100.0,0.0,0.0,101617.0979,0.193121,10.0,0.784799,...,1,1,1,1,1,1,1,1,1,3


In [ ]:
# Converting date_time column in date_time type data
# Creating a new dataframe with name train_data and inserting all the required columns

train["date_time"]=pd.to_datetime(train.date_time_IST)

train_data = pd.DataFrame()
train_data['date_time'] = train['date_time_IST']
train_data['temp']= train['tmpc']
train_data['dew'] = train['dwpc']
train_data['humidity'] = train['relh']
train_data['pressure'] = train['pressure']
train_data['visibility'] = train['vsbykm']

train_data["date_time"]=pd.to_datetime(train_data.date_time)

In [ ]:
train_data.shape

(78891, 6)

In [ ]:
# adding fog duration column
train_data["endtime"] = train_data["date_time"].shift(1)
train_data.loc[train_data.visibility<2.0,'fog_duration']=(train_data['date_time']-train_data['endtime']).astype('timedelta64[s]') #astype here to convert time to float sec
train_data.loc[train_data.visibility>=2.0,'fog_duration']=0

# adding energy_loss column
train_data.loc[train_data.visibility<2.0, 'energy_loss']=train_data['fog_duration']*(1-np.exp(-0.05*3/train_data['visibility']))
train_data.loc[train_data.visibility>=2.0,'energy_loss']=0

# adding fog_index column
train_data["fog_index"]= train_data["energy_loss"]/train_data["fog_duration"]
train_data.loc[train_data.fog_duration==0,'fog_index']=0

# dropping all the NaN Values
train_data.dropna(inplace = True)


In [ ]:
train_data.head(2)

,date_time,temp,dew,humidity,pressure,visibility,endtime,fog_duration,energy_loss,fog_index
1,2000-01-01 00:30:00,10.0,10.0,100.0,101693.2917,0.193121,2000-01-01 00:00:00,1800.0,972.15493,0.540086
2,2000-01-01 01:00:00,10.0,10.0,100.0,101693.2917,0.193121,2000-01-01 00:30:00,1800.0,972.15493,0.540086


In [ ]:
train_data.shape

(78890, 10)

Total number of rows is 78890 with 30 min interval data , so by doing resampling in 6 hr expected rows wiil be 78890/12 = **6574.25** which is roughly **6574 rows**.

In [ ]:
# resampled in 6h
# making datetime as index
train_data.set_index("date_time",inplace=True)
train_data_6h=pd.DataFrame()
train_data_6h["avg_air_temp"]=train_data["temp"].resample("6H",loffset="4h").mean()

In [ ]:
train_data_6h.shape

(24724, 1)

Expectation number of rows are much less than the actual one , this occurs because in our **dataset is not consistence with the time series** , there are missing values , so in resampled dataset some values are averaged of 12 (As Expected), 3 ,4 etc. **Even for some duration we don't have any data for resampling , for those time stamp we are getting NaN values**.

In [ ]:
train_data_6h.isna().sum()

avg_air_temp    17143
dtype: int64

Adding other columns in resampled 6 hour train dataset

In [ ]:
train_data_6h["avg_dew_point"]=train_data["dew"].resample("6h",loffset="4h").mean()

train_data_6h["avg_relative_humidity"] = train_data["humidity"].resample("6h",loffset="4h").mean()

train_data_6h["avg_pressure"] = train_data["pressure"].resample("6h",loffset="4h").mean()

train_data_6h["avg_visibility"] = train_data["visibility"].resample("6h",loffset="4h").mean()

train_data_6h["energy_loss"]=train_data.resample('6h',loffset="4h").agg({'energy_loss':'sum'}) #above and below will work as same

train_data_6h["fog_duration"]=train_data.resample('6h',loffset="4h").agg({'fog_duration':'sum'})

#Adding Fog_month variable
train_data_6h["fog_month"]=0
train_data_6h.loc[(train_data_6h.index.month==12)|(train_data_6h.index.month==1),"fog_month"]=1
train_data_6h.dropna(inplace = True)

# fog_index variable for 6 hours
train_data_6h['fog_index']=train_data_6h['energy_loss']/train_data_6h['fog_duration']
train_data_6h.loc[train_data_6h.fog_duration==0,'fog_index']=0

# The solution to tackle with NaN values we adopt here is to drop them .
train_data_6h.dropna(inplace = True)

In [ ]:
train_data_6h.shape

(7581, 9)

In [ ]:
train_data_6h.head()

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index
date_time,,,,,,,,,
2000-01-01 04:00:00,10.000000,10.000000,100.000000,101631.207878,0.193121,10693.704233,19800.0,1,0.540086
2000-01-01 10:00:00,10.860000,10.430000,97.136333,101849.065640,0.410382,8798.058547,21600.0,1,0.407318
2000-01-01 16:00:00,13.175000,11.625000,90.555000,101629.796892,1.425607,2349.704586,21600.0,1,0.108783
2000-01-01 22:00:00,9.250000,9.125000,98.656250,101722.922625,0.193121,11665.859163,21600.0,1,0.540086
2000-01-02 04:00:00,14.304545,10.927273,80.156818,101076.814791,2.980937,0.000000,0.0,1,0.000000


Now our preprocessed train dataset is ready so, we are saving it in Dataset folder.

In [ ]:
train_data_6h.to_csv('/content/drive/MyDrive/Dataset/Current dataset/train_preprocessed.csv')

Similarly, the preprocessing steps applies on test and validation dataset

In [ ]:
# for test dataset preprocessing

test = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Original /test_set.csv')
test["date_time"]=pd.to_datetime(test.date_time_IST)

test_data = pd.DataFrame()
test_data['date_time'] = test['date_time_IST']
test_data['temp']= test['tmpc']
test_data['dew'] = test['dwpc']
test_data['humidity'] = test['relh']
test_data['pressure'] = test['pressure']
test_data['visibility'] = test['vsbykm']
#coverting type of date_time column
test_data["date_time"]=pd.to_datetime(test_data.date_time)
# adding fog duration var
test_data["endtime"] = test_data["date_time"].shift(1)
test_data.loc[test_data.visibility<2.0,'fog_duration']=(test_data['date_time']-test_data['endtime']).astype('timedelta64[s]')#astype here to convert time to float sec
test_data.loc[test_data.visibility>=2.0,'fog_duration']=0
# adding energy_loss var
test_data.loc[test_data.visibility<2.0, 'energy_loss']=test_data['fog_duration']*(1-np.exp(-0.05*3/test_data['visibility']))
test_data.loc[test_data.visibility>=2.0,'energy_loss']=0
# fog index
test_data["fog_index"]= test_data["energy_loss"]/test_data["fog_duration"]
test_data.loc[test_data.fog_duration==0,'fog_index']=0
test_data.dropna(inplace = True)

# resampled in 6h
# making datetime as index
test_data.set_index("date_time",inplace=True)
test_data_6h=pd.DataFrame()
test_data_6h["avg_air_temp"]=test_data["temp"].resample("6H",loffset="4h").mean()

test_data_6h["avg_dew_point"]=test_data["dew"].resample("6h",loffset="4h").mean()

test_data_6h["avg_relative_humidity"] = test_data["humidity"].resample("6h",loffset="4h").mean()

test_data_6h["avg_pressure"] = test_data["pressure"].resample("6h",loffset="4h").mean()

test_data_6h["avg_visibility"] = test_data["visibility"].resample("6h",loffset="4h").mean()

test_data_6h["energy_loss"]=test_data.resample('6h',loffset="4h").agg({'energy_loss':'sum'}) #above and below will work as same

test_data_6h["fog_duration"]=test_data.resample('6h',loffset="4h").agg({'fog_duration':'sum'})

#Adding Fog_month variable
test_data_6h["fog_month"]=0
test_data_6h.loc[(test_data_6h.index.month==12)|(test_data_6h.index.month==1),"fog_month"]=1
test_data_6h.dropna(inplace = True)

# fog_index variable for 6 hours
test_data_6h['fog_index']=test_data_6h['energy_loss']/test_data_6h['fog_duration']
test_data_6h.loc[test_data_6h.fog_duration==0,'fog_index']=0
test_data_6h.dropna(inplace = True)


In [ ]:
test_data_6h.head(6)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index
date_time,,,,,,,,,
2019-11-01 04:00:00,19.000000,18.727273,98.338182,29.866364,0.481339,5492.121151,19800.0,0,0.277380
2019-11-01 10:00:00,22.333333,19.250000,83.659167,29.926250,0.838198,4137.855667,21600.0,0,0.191567
2019-11-01 16:00:00,29.791667,18.291667,50.613750,29.846667,1.929867,1626.558485,21600.0,0,0.075304
2019-11-01 22:00:00,23.166667,21.000000,87.858333,29.867500,1.351846,2292.035988,21600.0,0,0.106113
2019-11-02 04:00:00,19.833333,19.416667,97.496667,29.862500,0.777848,3812.834509,21600.0,0,0.176520
2019-11-02 10:00:00,23.333333,19.583333,81.206667,29.950000,1.255285,3028.665619,21600.0,0,0.140216


Here, preprocessed test dataset is ready so, we are saving it in Dataset folder.

In [ ]:
train_data_6h.to_csv('/content/drive/MyDrive/Dataset/Current dataset/test_preprocessed.csv')

In [ ]:
# for val dataset preprocessing

val = pd.read_csv('/content/drive/MyDrive/Dataset/Current dataset/Original /val_set.csv')
val["date_time"]=pd.to_datetime(val.date_time_IST)

val_data = pd.DataFrame()
val_data['date_time'] = val['date_time_IST']
val_data['temp']= val['tmpc']
val_data['dew'] = val['dwpc']
val_data['humidity'] = val['relh']
val_data['pressure'] = val['pressure']
val_data['visibility'] = val['vsbykm']
#coverting type of date_time column
val_data["date_time"]=pd.to_datetime(val_data.date_time)
# adding fog duration var
val_data["endtime"] = val_data["date_time"].shift(1)
val_data.loc[val_data.visibility<2.0,'fog_duration']=(val_data['date_time']-val_data['endtime']).astype('timedelta64[s]')#astype here to convert time to float sec
val_data.loc[val_data.visibility>=2.0,'fog_duration']=0
# adding energy_loss var
val_data.loc[val_data.visibility<2.0, 'energy_loss']=val_data['fog_duration']*(1-np.exp(-0.05*3/val_data['visibility']))
val_data.loc[val_data.visibility>=2.0,'energy_loss']=0
# fog index
val_data["fog_index"]= val_data["energy_loss"]/val_data["fog_duration"]
val_data.loc[val_data.fog_duration==0,'fog_index']=0
val_data.dropna(inplace = True)

# resampled in 6h
# making datetime as index
val_data.set_index("date_time",inplace=True)
val_data_6h=pd.DataFrame()
val_data_6h["avg_air_temp"]=val_data["temp"].resample("6H",loffset="4h").mean()

val_data_6h["avg_dew_point"]=val_data["dew"].resample("6h",loffset="4h").mean()

val_data_6h["avg_relative_humidity"] = val_data["humidity"].resample("6h",loffset="4h").mean()

val_data_6h["avg_pressure"] = val_data["pressure"].resample("6h",loffset="4h").mean()

val_data_6h["avg_visibility"] = val_data["visibility"].resample("6h",loffset="4h").mean()

val_data_6h["energy_loss"]=val_data.resample('6h',loffset="4h").agg({'energy_loss':'sum'}) #above and below will work as same

val_data_6h["fog_duration"]=val_data.resample('6h',loffset="4h").agg({'fog_duration':'sum'})

# Adding Fog_month variable
val_data_6h["fog_month"]=0
val_data_6h.loc[(val_data_6h.index.month==12)|(val_data_6h.index.month==1),"fog_month"]=1
val_data_6h.dropna(inplace = True)

# fog_index variable for 6 hours
val_data_6h['fog_index']=val_data_6h['energy_loss']/val_data_6h['fog_duration']
val_data_6h.loc[val_data_6h.fog_duration==0,'fog_index']=0
val_data_6h.dropna(inplace = True)



In [ ]:
val_data_6h.head(4)

,avg_air_temp,avg_dew_point,avg_relative_humidity,avg_pressure,avg_visibility,energy_loss,fog_duration,fog_month,fog_index
date_time,,,,,,,,,
2016-11-01 04:00:00,21.454545,16.227273,75.413182,29.957727,1.904873,1511.326285,19800.0,0,0.076330
2016-11-01 10:00:00,22.916667,15.833333,67.022500,30.015000,1.746134,1811.953348,21600.0,0,0.083887
2016-11-01 16:00:00,30.500000,13.750000,36.085833,29.927500,3.541889,0.000000,0.0,0,0.000000
2016-11-01 22:00:00,23.208333,14.375000,58.249167,29.935000,2.577626,0.000000,0.0,0,0.000000


Here, preprocessed validation dataset is ready so, we are saving it in Dataset folder.

In [ ]:
val_data_6h.to_csv('/content/drive/MyDrive/Dataset/Current dataset/val_preprocessed.csv')

## **Thank You**